In [ ]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

model = nn.Sequential(nn.Dropout(p=0.2),
                      nn.Linear(784, 256),
                      nn.ReLU(),
                      nn.Dropout(p=0.2),
                      nn.Linear(256, 128),
                      nn.ReLU(),
                      nn.Dropout(p=0.2),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)


epochs = 20
train_losses , test_losses = [], []
# model.cuda()
for e in range(epochs):
    training_loss = 0
    model.train()
    for images, labels in trainloader:
        
        images = images.view(images.shape[0], -1)
        
        optimizer.zero_grad()
        
        log_ps = model(images)
        
        loss = criterion(log_ps, labels)
        
        loss.backward()
        optimizer.step()
        
        training_loss += loss
    else:
        # Validation pass
        model.eval()
        testing_loss = 0
        accuracy = 0
        
        with torch.no_grad():
            for images, labels in testloader:
                images = images.view(images.shape[0], -1)
                
                log_ps = model(images)
                ps = torch.exp(log_ps)
                
                loss = criterion(log_ps, labels)
                testing_loss += loss
                
                top_p, top_classes = ps.topk(1, dim=1)
                
                eq = top_classes == labels.view(*top_classes.shape)
                
                accuracy += torch.mean(eq.type(torch.FloatTensor))
            else:
                train_l = len(trainloader)
                test_l = len(testloader)
                train_losses.append(training_loss/train_l)
                test_losses.append(testing_loss/test_l)
                print("Epoch: {}, Accuracy: {}, Training Loss: {}, Testing Loss: {}".format(e, accuracy/test_l, train_losses[-1], test_losses[-1]))

        

In [ ]:
import helper

# Test out your network!

model.eval()

dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]
# Convert 2D image to 1D vector
img = img.view(1, 784)

# Calculate the class probabilities (softmax) for img
with torch.no_grad():
    output = model(img)
#     output = model.forward(img)


ps = torch.exp(output)

# Plot the image and probabilities
helper.view_classify(img.view(1, 28, 28), ps, version='Fashion')